### Check GPU Availability in PyTorch

In [14]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU is available. Using: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")


GPU is available. Using: NVIDIA GeForce GTX 1080


### Test a Simple Tensor Operation on the GPU

In [15]:
# Create a tensor and move it to the GPU (if available)
tensor = torch.rand(3, 3).to(device)

# Check the device the tensor is stored on
print(f"Tensor is on: {tensor.device}")


Tensor is on: cuda:0


## Step-by-Step Code for Face Recognition in PyTorch

### Folder Structure

In [16]:
'''
dataset/
  ├── person1/
  │   ├── img1.jpg
  │   ├── img2.jpg
  │   └── ...
  └── person2/
      ├── img1.jpg
      ├── img2.jpg
      └── ...
'''

'\ndataset/\n  ├── person1/\n  │   ├── img1.jpg\n  │   ├── img2.jpg\n  │   └── ...\n  └── person2/\n      ├── img1.jpg\n      ├── img2.jpg\n      └── ...\n'

### Step 1: Import Required Libraries

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os


In [18]:
import torch
print(torch.__version__)


2.4.1+cu124


### Step 2: Data Preprocessing and Loading

In [19]:
# Define data transforms (resizing and normalizing)
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 (input size for most models)
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization values (used by pretrained models)
])

# Load the dataset from the 'dataset' folder
data_dir = 'E:/GitHub-rzn/poopy_or_paia/Dataset'
dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Split dataset into training and validation sets (80% training, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create DataLoader to load the images in batches
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)

# Get class names (i.e., the folder names 'person1' and 'person2')
class_names = dataset.classes
print(class_names)  # Output will be ['person1', 'person2']


['Prapty', 'Rezwan']


### Step 3: Define the Model

In [20]:
# Load a pretrained ResNet18 model
model = models.resnet18(pretrained=True)

# Freeze the parameters of the model so that we don't update them during training
for param in model.parameters():
    param.requires_grad = False

# Modify the final fully connected layer for 2 classes (person1 and person2)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


### Step 4: Define the Loss Function and Optimizer

In [21]:
# Define loss function (CrossEntropyLoss for classification)
criterion = nn.CrossEntropyLoss()

# Define optimizer (only for the last layer since others are frozen)
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


### Step 5: Training the Model

In [22]:
# Function to train the model
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    model.train()  # Set model to training mode

    for epoch in range(num_epochs):
        running_loss = 0.0

        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  # Zero the parameter gradients

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloaders['train'].dataset)
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    print('Training complete')

# Create a dictionary of dataloaders
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

# Train the model for 5 epochs
train_model(model, dataloaders, criterion, optimizer, num_epochs=7)


Epoch 1/7, Loss: 0.7236
Epoch 2/7, Loss: 0.5609
Epoch 3/7, Loss: 0.5145
Epoch 4/7, Loss: 0.5674
Epoch 5/7, Loss: 0.3978
Epoch 6/7, Loss: 0.3908
Epoch 7/7, Loss: 0.4112
Training complete


### Step 6: Evaluating the Model

In [23]:
# Function to evaluate the model
def evaluate_model(model, dataloaders):
    model.eval()  # Set model to evaluation mode
    corrects = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in dataloaders['val']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            corrects += torch.sum(preds == labels)

    accuracy = corrects.double() / len(dataloaders['val'].dataset)
    print(f'Validation Accuracy: {accuracy:.4f}')

# Evaluate the model
evaluate_model(model, dataloaders)


Validation Accuracy: 0.7647


### Step 7: Making Predictions

In [24]:
from PIL import Image

# Function to predict the class of a single image
def predict_image(image_path, model):
    model.eval()

    # Load the image
    image = Image.open(image_path)
    image = data_transforms(image).unsqueeze(0)  # Transform and add batch dimension
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    return class_names[preds[0]]

# Test the model with a new image
image_path = 'C:/Users/rznis/OneDrive/Desktop/Desktop/4.png'
prediction = predict_image(image_path, model)
print(f'Predicted Class: {prediction}')


Predicted Class: Rezwan


In [25]:
import os
import torch

# Define the path to the model directory and model file
model_dir = 'model'
model_save_path = os.path.join(model_dir, 'poopy-or-paia-pytorch.pth')

# Create the model directory if it does not exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Assuming `model` is your trained PyTorch model
# Save the model state_dict (recommended)
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to model\poopy-or-paia-pytorch.pth
